[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/pinecone/sparse/bm25/bm25-vector-generation.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/pinecone/sparse/bm25/bm25-vector-generation.ipynb)

# Hybrid Search with BM25 Sparse Vectors


## Overview

BM25 is a popular technique for retrieving text. It uses term frequencies to determine the relative importance of the term to the query. It is simple but effective and only requires knowing the number of documents in a corpus and the frequency of terms across documents. In the following guide, we will show how to use BM25 with Pinecone's sparse-dense vectors for use in hybrid search.

Skip the embedding creation step by using the [companion guide](https://github.com/pinecone-io/examples/blob/master/pinecone/sparse/bm25/bm25-quora.ipynb).

## Prerequisites

We'll install the required libraries:

In [1]:
!pip install -qU \
          torch \
          sentence-transformers \
          pinecone-text \
          scikit-learn \
          pandas \
          pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 19.5 MB/s eta 0:00:00


In [5]:
from tqdm.notebook import tqdm

## Quora Dataset

We'll load the popular Quora dataset:

In [3]:
import pandas as pd

df = pd.read_parquet("https://storage.googleapis.com/pinecone-datasets-dev/quora_all-MiniLM-L6-bm25/raw/quora_questions_sample.parquet")

In [4]:
df.head()

,id,text
0,17248,"If I fall under the Brady law due to PTSD, is..."
1,240419,Which question can't be answered with a yes o...
2,262372,How can I write a children's book for older k...
3,180057,What happens when you view a public Instagram...
4,456610,What is the fact about NIBIRU the Planet X?


### Fit BM25 with Spacy Tokenizer

We'll create fit a BM25 model using a simple split by space tokenization.


In [7]:
from pinecone_text.sparse import BM25

bm25 = BM25(tokenizer=lambda s: s.split())

We need to calculate how often tokens appear in documents for BM25 to be able to create sparse vectors. To do this we call `bm25.fit` across our full dataset.

In [8]:
bm25.fit(df['text'])


### Dense Model

We use the popular all-MiniLM-L6-v2 model available on Hugging Face for dense vectors.

In [9]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"running on {device}")

model = SentenceTransformer(
    'sentence-transformers/all-MiniLM-L6-v2',
    device=device
)

running on cpu


### Compute Dense & Sparse Embeddings

Create BM25 sparse embeddings:

In [12]:
df['sparse_values'] = df['text'].apply(bm25.encode_document)

And now encode dense vector embeddings:

In [13]:
batch_size = 128
dense_values = []
for i in tqdm(range(0, len(df), batch_size)):
  dense_values += model.encode(df.iloc[i:i + batch_size]["text"].tolist()).tolist()

df['values'] = dense_values

  0%|          | 0/79 [00:00<?, ?it/s]

We organize our dataframe to align to the `pinecone-datasets` format:

In [14]:
df_result = df.copy()
df_result["metadata"] = None
df_result["blob"] = df_result["text"].apply(lambda t: {"text": t})
df_result = df_result.drop(columns="text")

In [15]:
df_result.head()

,id,sparse_values,values,metadata,blob
0,17248,"{'indices': [23236, 24734, 29533, 34988, 42257...","[0.021123135462403297, 0.043918054550886154, -...",None,{'text': ' If I fall under the Brady law due t...
1,240419,"{'indices': [38437, 44017, 70189, 71832, 92594...","[0.015179850161075592, 0.06904047727584839, -0...",None,{'text': ' Which question can't be answered wi...
2,262372,"{'indices': [15630, 23579, 31925, 92594, 10235...","[0.038049325346946716, 0.08449698239564896, 0....",None,{'text': ' How can I write a children's book f...
3,180057,"{'indices': [30500, 45980, 50743, 92594, 13752...","[-0.028053421527147293, -0.04008297994732857, ...",None,{'text': ' What happens when you view a public...
4,456610,"{'indices': [9251, 21662, 24734, 70299, 76665,...","[0.0056223683059215546, 0.10487581044435501, 0...",None,{'text': ' What is the fact about NIBIRU the P...


And now we have all we need to start using Pinecone vector database 🚀

For more details on that, check out [this notebook](https://github.com/pinecone-io/examples/blob/master/pinecone/sparse/bm25/bm25-quora.ipynb).